In [1]:
import os
import requests
from bs4 import BeautifulSoup as bs
import boto3

In [2]:
# Configuração inicial
url = 'https://www.gov.br/inep/pt-br/acesso-a-informacao/dados-abertos/microdados/censo-da-educacao-superior'

s3 = boto3.client('s3')
bucket_name = 'elasticbeanstalk-us-east-2-047710767346'
subfolder_name = 'Case_A3'

# verificar se a pasta Case_A3 existe no bucket
try:
    s3.head_object(Bucket=bucket_name, Key=subfolder_name + '/')
except:
    # se a pasta não existe, criar
    s3.put_object(Bucket=bucket_name, Key=subfolder_name + '/')

# verificar se a pasta "arquivos_zip" dentro de Case_A3 existe no bucket
try:
    s3.head_object(Bucket=bucket_name, Key=f'{subfolder_name}/arquivos_zip/')
except:
    # se a pasta não existe, criar
    s3.put_object(Bucket=bucket_name, Key=f'{subfolder_name}/arquivos_zip/')

In [3]:
# raspando a página
html_bruto = requests.get(url, verify=False) 
html_conteudo = html_bruto.text
html_estruturado = bs(html_bruto.content, 'html.parser')

# Configurando o parser e baixando os dados
html_estruturado = bs(html_bruto.content, 'html.parser')

# separando a div desejada - "parent_div"
parent_div = html_estruturado.find('div', {'id': 'parent-fieldname-text'})

# lista para armazenar os links para download
links_download = []

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.gov.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [4]:
# percorrendo os elementos da div para encontrar os links

for year_element in parent_div.find_all('p', {'class': 'callout'}):
    year = year_element.find('strong').text.strip()
    ul_element = year_element.find_next('ul')

    for li_element in ul_element.find_all('li'):
        link = li_element.find('a')['href']
        links_download.append((year, link))

In [5]:
# para impedir que a mensagem de erro apareça

from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [6]:
# itera sobre os links para fazer o download e salvar os arquivos
for year, link in links_download:
    try:
        # faz o download e desativa a verificação do certificado SSL
        arquivo_zip = requests.get(link, stream=True, verify=False)
        arquivo_zip.raise_for_status()  # Verifica se houve erro no download

        # envia o arquivo para o bucket do S3 dentro da subpasta "arquivos_zip"
        s3.upload_fileobj(
            arquivo_zip.raw,  # fileobj é o objeto do arquivo a ser enviado. raw é o conteúdo do arquivo e indica que o arquivo não foi aberto (camada de abstração)
            bucket_name,  # nome do bucket
            f'{subfolder_name}/arquivos_zip/{year}_{link.split("/")[-1]}'  # nome do arquivo no bucket (ano_nome_do_arquivo.zip) e a subpasta (arquivos_zip)
        )

        print(f'O arquivo {year}_{link.split("/")[-1]} foi enviado para o bucket na subpasta {subfolder_name}/arquivos_zip.')

    except Exception as e:
        print(f'Erro ao baixar o arquivo {year}_{link.split("/")[-1]}: {e}')

O arquivo 2022_microdados_censo_da_educacao_superior_2022.zip foi enviado para o bucket na subpasta Case_A3/arquivos_zip.
O arquivo 2021_microdados_censo_da_educacao_superior_2021.zip foi enviado para o bucket na subpasta Case_A3/arquivos_zip.
O arquivo 2020_microdados_censo_da_educacao_superior_2020.zip foi enviado para o bucket na subpasta Case_A3/arquivos_zip.
O arquivo 2019_microdados_censo_da_educacao_superior_2019.zip foi enviado para o bucket na subpasta Case_A3/arquivos_zip.
O arquivo 2018_microdados_censo_da_educacao_superior_2018.zip foi enviado para o bucket na subpasta Case_A3/arquivos_zip.
O arquivo 2017_microdados_censo_da_educacao_superior_2017.zip foi enviado para o bucket na subpasta Case_A3/arquivos_zip.
O arquivo 2016_microdados_censo_da_educacao_superior_2016.zip foi enviado para o bucket na subpasta Case_A3/arquivos_zip.
O arquivo 2015_microdados_censo_da_educacao_superior_2015.zip foi enviado para o bucket na subpasta Case_A3/arquivos_zip.
O arquivo 2014_microdado